## Setting the simulation

All the data needed to run this notebook is here [test-data](test-data/)

In [1]:
import pandas as pd
import gzip
import time
import numpy as np
import fwdpy11
import msprime
fwdpy11.__version__

'0.17.2.dev15+g84f4f8ed'

## Loading the data

I have generated 200 random samples of 1Mb length from the genome. Here, I take one of those samples, as an example, to set the simulation.
**NOTE:**

- I subtract the start position of the region from the intervals and the recombination map. So, the start position is zero. (Not sure If I need to do this).

In [2]:
# The 1Mb sampled region

chromosome, start, end = np.loadtxt('test-data/region_region_23.bed', dtype=np.int64)
print(f'position chr{chromosome}, {start}, {end}')

position chr22, 29000000, 30000000


In [3]:
# exonic intervals in the sampled region

exons = pd.read_csv('test-data/region_exons_23.bed', sep='\t', names=['chro', 'start', 'end'])
exons.head()

,chro,start,end
0,22,29042494,29042569
1,22,29043298,29043430
2,22,29044779,29044890
3,22,29046715,29046883
4,22,29048388,29048491


In [4]:
# non coding intervals (intronic and exonic) in the sampled region
nonexonic = pd.read_csv('test-data/region_intronANDinterg_23.bed', sep='\t', names=['chro', 'start', 'end'])
nonexonic.head()

,chro,start,end
0,22,29000000,29042494
1,22,29042569,29043298
2,22,29043430,29044779
3,22,29044890,29046715
4,22,29046883,29048388


In [5]:
# mutation rates within the region
ml_coding = pd.read_csv('test-data/region_mlcoding_23.csv')
ml_non_coding = np.loadtxt('test-data/region_mlnoncoding_23.txt', dtype=np.object0)[1]
ml_non_coding = float(ml_non_coding)

## Create a dict mapping class name to value

ml = {}
for x, y in ml_coding.iterrows():
    ml[y.Q] = y.mL

ml['noncoding'] = ml_non_coding
print(ml)

{'LOF': 1.151627658e-05, 'missense': 0.00019355650402, 'synonymous': 7.921106974e-05, 'noncoding': 0.00957497053154}


In [6]:
# substract start position so the intial positon is zero
exons['start'] = exons['start'] - start
exons['end'] = exons['end'] - start

nonexonic['start'] = nonexonic['start'] - start
nonexonic['end'] = nonexonic['end'] - start

nonexonic.head()

,chro,start,end
0,22,0,42494
1,22,42569,43298
2,22,43430,44779
3,22,44890,46715
4,22,46883,48388


In [7]:
## Recombination map
## Here I use msprime function: msprime.RateMap.read_hapmap to load the recombination map
rmap = msprime.RateMap.read_hapmap('test-data/chr22.b38.gmap', position_col=0, map_col=2)
print(rmap)


┌───────────────────────────────────────────────────────┐
│left      │right     │         mid│      span│     rate│
├───────────────────────────────────────────────────────┤
│0         │15287922  │     7643961│  15287922│  9.5e-10│
│15287922  │16370978  │    15829450│   1083056│  1.2e-09│
│16370978  │16372046  │    16371512│      1068│    5e-09│
│16372046  │16373044  │    16372545│       998│    5e-09│
│16373044  │16373740  │    16373392│       696│  5.1e-09│
│16373740  │16374612  │    16374176│       872│    6e-09│
│16374612  │16374641  │  16374626.5│        29│  2.7e-08│
│16374641  │16374814  │  16374727.5│       173│  2.7e-08│
│16374814  │16374956  │    16374885│       142│  8.6e-09│
│16374956  │16375126  │    16375041│       170│  4.2e-09│
│⋯         │⋯         │           ⋯│         ⋯│        ⋯│
│50739662  │50744589  │  50742125.5│      4927│  6.5e-09│
│50744589  │50757736  │  50751162.5│     13147│  6.4e-09│
│50757736  │50761423  │  50759579.5│      3687│  6.4e-09│
│50761423  │5

In [8]:
## we can take a slice from the map to get the coordinates in the sampled region
# with set trim=True 
rmap = rmap.slice(left=start, right=end, trim=True)
print(rmap)


┌──────────────────────────────────────────────┐
│left    │right    │       mid│  span│     rate│
├──────────────────────────────────────────────┤
│0       │2389     │    1194.5│  2389│  1.4e-09│
│2389    │2471     │      2430│    82│  9.3e-09│
│2471    │4331     │      3401│  1860│  1.5e-08│
│4331    │4527     │      4429│   196│  8.4e-09│
│4527    │5345     │      4936│   818│  1.3e-08│
│5345    │6551     │      5948│  1206│  9.5e-10│
│6551    │6763     │      6657│   212│  1.1e-09│
│6763    │6844     │    6803.5│    81│  9.9e-10│
│6844    │7062     │      6953│   218│  1.3e-09│
│7062    │8663     │    7862.5│  1601│  3.5e-09│
│⋯       │⋯        │         ⋯│     ⋯│        ⋯│
│994559  │994665   │    994612│   106│        0│
│994665  │994787   │    994726│   122│        0│
│994787  │995001   │    994894│   214│  4.7e-11│
│995001  │995292   │  995146.5│   291│  3.4e-11│
│995292  │996551   │  995921.5│  1259│    4e-11│
│996551  │997514   │  997032.5│   963│  3.1e-11│
│997514  │997555   

## Compute m (per base mutation rate)

In [9]:

def per_base_mutation_rate(mL, regions):
    L = (regions.end - regions.start).sum()
    return mL / L

u = dict()
u['noncoding'] = per_base_mutation_rate(ml['noncoding'], regions=nonexonic)

# coding
u['synonymous'] = per_base_mutation_rate(ml['synonymous'], regions=exons)
u['missense'] = per_base_mutation_rate(ml['missense'], regions=exons)
u['LOF'] = per_base_mutation_rate(ml['LOF'], regions=exons)

for x in u:
    print(f'{x}: {u[x]}')

noncoding: 1.0001243533421491e-08
synonymous: 2.7431455097658955e-09
missense: 6.703023411137277e-09
LOF: 3.988182774622524e-10


# Set simulation 

## Neutral regions

In [10]:
## we will label the mutations according to the functional category

mut_labels = {
    'neutral': 0,
    'missense': 1,
    'synonymous': 2,
    'LOF': 3,
}

In [11]:
nonexonic.head()

,chro,start,end
0,22,0,42494
1,22,42569,43298
2,22,43430,44779
3,22,44890,46715
4,22,46883,48388


In [12]:
# Construct the neutral regions from the non-exonic intervals
# we also assume that synonymous mutations are neutral

nregions = []
for _, noexon in nonexonic.iterrows():
    nregions.append(
        fwdpy11.Region(beg=noexon.start, end=noexon.end, weight=u['noncoding'], label=mut_labels['neutral'])
    
    )

# synonymous we assume they are neutral
for _, exon in exons.iterrows():
    nregions.append(
        fwdpy11.Region(beg=exon.start, end=exon.end, weight=u['synonymous'], label=mut_labels['synonymous'])
    
    )

## Distributions of effect sizes | Selected regions

- For now I use Aaron's infered DFEs [see here](https://moments.readthedocs.io/en/main/modules/dfe.html#all-data).
- The weights establish the relative probability that a mutation comes from a given region.

**NOTE:**

- When multiple “sregion” objects are used, the default behavior is to multiply the input weight by end-beg:
- The weights should depend on the mutation type (i.e. synonymous, missense). We could make the weight
proportional to ml.

**Comments:**

- The selection and dominance should also depend on the mutation class. We'll need to pick an appropiate DFEs for each case.


### DFE for missense variants

The parameters that were fit are alpha and beta (or shape and scale) of the gamma distribution.

- Ne = 11372.91
- shape: 0.1596
- scale: 2332.3

The mean of the gamma distribution is $\alpha\beta$. I need to divide by 2Ne.


In [13]:
Ne = 11372.91
shape = 0.1596
scale = 2332.3
mean_s = (shape * scale) / (2 * Ne)
mean_s

0.01636498838028262

In [14]:
# This will be the DFE for missense variants
fwdpy11.GammaS(beg=0, end=1, weight=1, mean=mean_s, shape_parameter=shape, h=1)

fwdpy11.GammaS(beg=0, end=1, weight=1, mean=0.01636498838028262, shape_parameter=0.1596, h=1, coupled=True, label=0, scaling=1.0)

In [15]:
# DFE for LOF
shape_lof = 0.3589
scale_lof = 7830.5
mean_s_lof = (shape_lof * scale_lof) / (2 * Ne)
mean_s_lof

0.1235552927966545

In [16]:
fwdpy11.GammaS(beg=0, end=1, weight=1, mean=mean_s_lof, shape_parameter=shape_lof, h=1)

fwdpy11.GammaS(beg=0, end=1, weight=1, mean=0.1235552927966545, shape_parameter=0.3589, h=1, coupled=True, label=0, scaling=1.0)

In [17]:
# Define the Weights
# I think we can interpret these weights as given that
# we have a mutation in a coding region what is the 
# probability that this mutation is missense

total_weigth = ml['missense'] + ml['LOF']


w_mis = ml['missense'] / total_weigth
w_lof = ml['LOF'] / total_weigth

print(f'total weight: {total_weigth}\n\nmissense={w_mis}\nlof={w_lof}')

total weight: 0.0002050727806

missense=0.943842978349902
lof=0.05615702165009801


In [18]:
# Construct the selected regions from the exonic intervals
sregions = []
for _, exon in exons.iterrows():
    # missense
    sregions.append(
        fwdpy11.GammaS(
            beg=exon.start, end=exon.end,
            weight=u['missense'],
            mean=mean_s, shape_parameter=shape,
            h=1,
            label=mut_labels['missense'])
    
    )
    # loss of function
    sregions.append(
        fwdpy11.GammaS(
            beg=exon.start, end=exon.end,
            weight=u['LOF'],
            mean=mean_s_lof, shape_parameter=shape_lof,
            h=1,
            label=mut_labels['LOF'])
    
    )

In [19]:
sregions[:5]

[fwdpy11.GammaS(beg=42494, end=42569, weight=6.703023411137277e-09, mean=0.01636498838028262, shape_parameter=0.1596, h=1, coupled=True, label=1, scaling=1.0),
 fwdpy11.GammaS(beg=42494, end=42569, weight=3.988182774622524e-10, mean=0.1235552927966545, shape_parameter=0.3589, h=1, coupled=True, label=3, scaling=1.0),
 fwdpy11.GammaS(beg=43298, end=43430, weight=6.703023411137277e-09, mean=0.01636498838028262, shape_parameter=0.1596, h=1, coupled=True, label=1, scaling=1.0),
 fwdpy11.GammaS(beg=43298, end=43430, weight=3.988182774622524e-10, mean=0.1235552927966545, shape_parameter=0.3589, h=1, coupled=True, label=3, scaling=1.0),
 fwdpy11.GammaS(beg=44779, end=44890, weight=6.703023411137277e-09, mean=0.01636498838028262, shape_parameter=0.1596, h=1, coupled=True, label=1, scaling=1.0)]

## Recombination

In [20]:
rmap

left,right,mid,span,rate
0,2389,1194.5,2389,1.4e-09
2389,2471,2430,82,9.3e-09
2471,4331,3401,1860,1.5e-08
4331,4527,4429,196,8.4e-09
4527,5345,4936,818,1.3e-08
5345,6551,5948,1206,9.5e-10
6551,6763,6657,212,1.1e-09
6763,6844,6803.5,81,9.9e-10
6844,7062,6953,218,1.3e-09
7062,8663,7862.5,1601,3.5e-09


In [21]:
nrec = len(rmap) - 1

**NOTE:** I think the recombination rate in the map above
    is per base pair

In [22]:
recregions = []
for i in range(nrec):
    recregions.append(
     fwdpy11.PoissonInterval(
         beg=rmap.left[i],
         end=rmap.right[i],
         mean=rmap.rate[i] * rmap.span[i]
     )   
    )

In [23]:
recregions[:10]

[fwdpy11.PoissonInterval(beg=0.0, end=2389.0, mean=3.373172757461935e-06, discrete=False),
 fwdpy11.PoissonInterval(beg=2389.0, end=2471.0, mean=7.599999999885475e-07, discrete=False),
 fwdpy11.PoissonInterval(beg=2471.0, end=4331.0, mean=2.8360000000060556e-05, discrete=False),
 fwdpy11.PoissonInterval(beg=4331.0, end=4527.0, mean=1.6399999999694439e-06, discrete=False),
 fwdpy11.PoissonInterval(beg=4527.0, end=5345.0, mean=1.0490000000029642e-05, discrete=False),
 fwdpy11.PoissonInterval(beg=5345.0, end=6551.0, mean=1.139999999955066e-06, discrete=False),
 fwdpy11.PoissonInterval(beg=6551.0, end=6763.0, mean=2.3000000004547158e-07, discrete=False),
 fwdpy11.PoissonInterval(beg=6763.0, end=6844.0, mean=7.999999995789153e-08, discrete=False),
 fwdpy11.PoissonInterval(beg=6844.0, end=7062.0, mean=2.800000000191538e-07, discrete=False),
 fwdpy11.PoissonInterval(beg=7062.0, end=8663.0, mean=5.580000000005025e-06, discrete=False)]

## Total remcombination rate

Is the total recombination rate the sum if the rate of each region?

In [24]:
total_recombination_rate = sum([x.mean for x in recregions])

print(f'total_recombination_rate is {total_recombination_rate}')

total_recombination_rate is 0.007796313172757488


## Rates

We need to specify the total rates

In [25]:
#  The neutral mutation rate, selected mutation rate, and total recombination rate, respectively.
neutral_ml = ml['noncoding'] + ml['synonymous']
selected_ml = ml['missense'] + ml['LOF']

# recomb_rate = ??? | I'm not sure how to set this value
rates = fwdpy11.MutationAndRecombinationRates(
    neutral_mutation_rate=neutral_ml,
    selected_mutation_rate=selected_ml,
    recombination_rate=None)


## Demography

To test the DFE I will use a constant size population model, this will run faster.

In [26]:
Ne = 5000
pop = fwdpy11.DiploidPopulation(N=Ne, length=int(1e6))
pop.N
pop.tables.genome_length


1000000.0

## Setting up the parameters for a simulation


In [27]:
SIM_LEN = 1 * pop.N #TODO: should be 10

In [28]:
# the parameters that fwdpy11 needs to run the simulation
p = {
    "nregions": nregions,  # neutral mutations (none for now, can add after the fact)
    "gvalue": fwdpy11.Additive(2.0),  # fitness model
    "sregions": sregions, 
    "recregions": recregions,
    "rates": rates,
    "prune_selected": True,
    "demography": fwdpy11.DiscreteDemography(),  # pass the demographic model
    "simlen": SIM_LEN
}

In [29]:
params = fwdpy11.ModelParams(**p)

In [30]:
# run the simulation
# set up the random number generator
rng = fwdpy11.GSLrng(54321) 

In [31]:
# run the simulation
print('runnning simulation ...')
time1 = time.time()
fwdpy11.evolvets(
    rng, pop, params, simplification_interval=100, suppress_table_indexing=True
)
print("Simulation took", int(time.time() - time1), "seconds")

# simulation finished
print("Final population sizes =", pop.deme_sizes())

runnning simulation ...
Simulation took 118 seconds
Final population sizes = (array([0], dtype=int32), array([500]))


In [32]:
mkdir -p results

In [33]:
# save the simulation results
with gzip.open('results/sim-pop.gz', 'wb') as f:
    pop.pickle_to_file(f)

## Results

- I will get the SFS for selected and not-selected mutations for 100 samples.

In [34]:
nodes = np.array(pop.tables.nodes, copy=False)
alive_nodes = pop.alive_nodes
deme0_nodes = alive_nodes[np.where(nodes["deme"][alive_nodes] == 0)[0]]

In [35]:
# SFS (neutral + selected mutations)
pop.tables.fs([deme0_nodes[:100]], include_neutral=True)

masked_array(data=[--, 23, 13, 5, 2, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0,
                   2, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, --],
             mask=[ True, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False,

In [36]:
# SFS selected
pop.tables.fs([deme0_nodes[:100]], include_neutral=False)

masked_array(data=[--, 3, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 1, 0, 0, 0, --],
             mask=[ True, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, False, False, False,
                   False, False, False, False, False, F

In [37]:
ts = pop.dump_tables_to_tskit()
ts

# How many mutations for each class do we observe?

In [38]:
from collections import Counter
mut_clas = []
for m in ts.mutations():
    mut_clas.append(m.metadata['label'])
    
Counter(mut_clas)

Counter({0: 90, 3: 2, 1: 7, 2: 1})